In [ ]:
import os
import urllib

import branca
import folium
import geopandas as gpd
import gspread
import pandas as pd
import requests
from flask import Flask, jsonify, render_template, request
from folium import plugins
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials

# Flaskアプリケーションのセットアップ
app = Flask(__name__)

# 認証情報の取得
scope = ["https://www.googleapis.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(
    "keys.json", scopes=scope
)

# スプレッドシートにアクセス
gClient = gspread.authorize(credentials)
SPREADSHEET_KEY = "11ajawzcNAp2hoq5897aaP2kCBYw2I7ZEDB0CK-mNapQ"
file = gClient.open_by_key(SPREADSHEET_KEY)

# worksheet = file.get_worksheet(0)
# data = pd.DataFrame(worksheet.get_all_records())

# 指定したシート名
sheet_names = ["新築戸建て", "中古戸建て", "新築マンション", "中古マンション"]
# 各シートをデータフレームに変換
data = {}
for sheet_name in sheet_names:
    sheet = file.worksheet(sheet_name)
    data[sheet_name] = pd.DataFrame(sheet.get_all_records())


# ルートエンドポイントの作成:
# データをHTML形式に整形して返す
# ルートエンドポイントの作成
@app.route("/")
def show_data():
    # 全てのデータをHTML形式に整形して返す
    htmls = [f"<h1>{sheet_name}</h1>{df.to_html()}" for sheet_name, df in data.items()]
    return "<br>".join(htmls)


@app.route("/search_form")
def search_form():
    return render_template("search_form.html")


@app.route("/search", methods=["GET", "POST"])
def search():
    new_or_used = request.form.get("new_or_used")
    house_or_mansion = request.form.get("house_or_mansion")
    price_range = request.form.get("price_range")

    selected_df = data[f"{new_or_used}{house_or_mansion}"]

    selected_df["販売価格"] = (
        selected_df["販売価格"].str.extract(r"(\d+)").fillna(0).astype(int)
    )

    min_price, max_price = (
        map(int, price_range.split("-"))
        if "-" in price_range
        else (int(price_range), None)
    )
    if max_price:
        results_df = selected_df[
            (selected_df["販売価格"] >= min_price) & (selected_df["販売価格"] <= max_price)
        ]
    else:
        results_df = selected_df[selected_df["販売価格"] >= min_price]

    # 新築戸建ての緯度経度取得
    latitude_list = []
    longitude_list = []

    name_list = results_df["物件名"].tolist()
    price_list = results_df["販売価格"].tolist()
    plan_list = results_df["間取り"].tolist()
    area_list = results_df["建物面積"].tolist()
    URL_list = results_df["リンク"].tolist()
    address_list = results_df["所在地"].tolist()

    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    for address in address_list:
        res = requests.get(url + urllib.parse.quote(address))
        latitude_list.append(res.json()[0]["geometry"]["coordinates"][1])
        longitude_list.append(res.json()[0]["geometry"]["coordinates"][0])

    results = []
    for i in range(len(results_df)):
        result = {
            "物件名": name_list[i],
            "販売価格": price_list[i],
            "間取り": plan_list[i],
            "建物面積": area_list[i],
            "リンク": URL_list[i],
            "所在地": address_list[i],
            "緯度": latitude_list[i],
            "経度": longitude_list[i],
        }
        results.append(result)

    # マップにプロット
    map_obj = folium.Map(location=[latitude_list[0], longitude_list[0]], zoom_start=15)

    for i in range(len(results)):
        popup_content = (
            f"<div style='writing-mode: horizontal-tb; white-space: nowrap;'>"
        )
        popup_content += f"{results[i]['物件名']}<br>"
        popup_content += f"販売価格: {results[i]['販売価格']}<br>"
        popup_content += f"間取り: {results[i]['間取り']}<br>"
        popup_content += f"建物面積: {results[i]['建物面積']}<br>"
        popup_content += f'<a href="{results[i]["リンク"]}" target="_blank">詳細</a>'
        popup_content += "</div>"

        iframe = branca.element.IFrame(html=popup_content, width=300, height=160)
        popup = folium.Popup(iframe, max_width=300)

        folium.Marker(
            location=[results[i]["緯度"], results[i]["経度"]], popup=popup
        ).add_to(map_obj)

        map_obj.save(os.path.join(app.root_path, "static", "map.html"))

        # マップをHTMLファイルとして保存

    return render_template(
        "search_results.html",
        results=results,
        map_path="map.html",
    )